In [67]:
import pandas as pd
import torch

df = pd.read_csv("../data/obs_small.csv")

In [68]:
model = torch.hub.load("pytorch/vision:v0.10.0", "mobilenet_v2", pretrained=True)

Using cache found in /Users/kaiose/.cache/torch/hub/pytorch_vision_v0.10.0


In [69]:
labels = {"Plantae": 0, "Insecta": 1, "Aves": 2, "Mammalia": 3}
label_weights = df["iconic_taxon_name"].value_counts().values.tolist()
class_weights = torch.tensor(label_weights, dtype=torch.float)
df["taxon_label"] = df["iconic_taxon_name"].map(labels)

In [70]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import models, transforms
from torch import nn, optim
from PIL import Image

# import requests


class CustomImageDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        label = self.dataframe.iloc[idx, 1]
        img_path = self.dataframe.iloc[idx, 0]
        image = Image.open(f"../data/images/{img_path}.jpg").convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label)


transform = transforms.Compose(
    [
        transforms.Resize((224, 224)),  # MobileNet input size is 224x224
        transforms.ToTensor(),  # Convert image to a PyTorch tensor
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize
    ]
)

df = df[["id", "taxon_label"]]


dataset = CustomImageDataset(dataframe=df, transform=transform)


train_size = int(0.8 * len(dataset))  # 80% for training
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])


dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

model = models.mobilenet_v2(pretrained=True)

num_classes = df["taxon_label"].nunique()  # Get the number of unique classes

# Modify the final fully connected layer (classifier) of MobileNetV2
model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(dataloader)
    accuracy = 100 * correct / total

    model.eval()  # Set model to evaluation mode
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():  # No need to calculate gradients for validation
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)  # Forward pass
            loss = criterion(outputs, labels)  # Calculate loss

            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_loss = val_loss / len(val_loader)
    val_accuracy = 100 * val_correct / val_total

    # Print results for the current epoch
    print(f"Epoch [{epoch+1}/{num_epochs}]")
    print(f"Train Loss: {loss:.4f}, Train Accuracy: {accuracy:.2f}%")
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")


print("Finished Fine-tuning the model")
torch.save(model.state_dict(), "../data/mobilenetft.pth")

Epoch [1/5]
Train Loss: 2.6568, Train Accuracy: 63.12%
Validation Loss: 1.4753, Validation Accuracy: 65.00%
Epoch [2/5]
Train Loss: 0.9205, Train Accuracy: 77.50%
Validation Loss: 0.7979, Validation Accuracy: 76.25%
Epoch [3/5]
Train Loss: 1.5742, Train Accuracy: 82.19%
Validation Loss: 0.7990, Validation Accuracy: 71.25%
Epoch [4/5]
Train Loss: 0.9657, Train Accuracy: 90.62%
Validation Loss: 0.7155, Validation Accuracy: 75.00%
Epoch [5/5]
Train Loss: 1.2883, Train Accuracy: 93.12%
Validation Loss: 1.0123, Validation Accuracy: 73.75%
Finished Fine-tuning the model
